# K Means POC experiment notebook 

## Experiment description
the notebook train simple m kmeans model. Model implemented  complying the ML framework



##### Jupyter helpers:

In [ ]:
%reload_ext autoreload
%autoreload

Define imports

In [ ]:

from kmeans_poc.data import KMeansSampleDataLoader
from kmeans_poc.models import KMeansModel
from kmeans_poc.data_processing import DataProcessor
from kmeans_poc.experimentation import MlflowExperimentation
from kmeans_poc.evaluation import Evaluator, EvaluationMetrics


## Load data
*replace MyDataLoader with your DataLoader implementation*

In [ ]:
data_loader = KMeansSampleDataLoader(dataset_name="generated",dataset_version=1.0 )
x = data_loader.get_dataset()

Define experimentation object, which will be used for logging the experiments parameters, metrics and artifacts
*Replace MlflowExperimentation if you use a different experimentation system*

In [ ]:
experimentation = MlflowExperimentation(tracking_uri="databricks")

Create model/logic:
Note: used default empty preprocessor, as there is no preprocessing for this model

In [ ]:
my_model = KMeansModel()

Define evaluation

In [ ]:
class MyEvaluator(Evaluator):
    def evaluate(self, **kwargs) -> EvaluationMetrics:
        pass

evaluator = MyEvaluator()

from kmeans_poc.evaluation import EvaluationMetrics, Evaluator


class KMeansSampleEvaluationMetrics(EvaluationMetrics):
    """
    Class to hold the actual values the evaluation created, e.g. precision, recall, MSE.
    in this sample case number of clusters is used to showcase the usage of framework only
    """
    def __init__(self, n_clusters):
        self.n_clusters = n_clusters
        super().__init__()

    def get_metrics(self):
        return {"n_clusters": self.n_clusters}
    
    def __repr__(self):
        return f"n_clusters: {self.n_clusters}"


class KMeansSampleEvaluator(Evaluator):
    """
    Class to hold the logic for how the model is evaluated.
    """
    def __init__(self, expected_n_clusters):
        self.exp_n_clusters = expected_n_clusters
        super().__init__()

    def evaluate(self, predicted, actual) -> KMeansSampleEvaluationMetrics:
        # This is where actual evaluation takes place.
        return KMeansSampleEvaluationMetrics(
            n_clusters = self.exp_n_clusters
        )


evaluator = KMeansSampleEvaluator(expected_n_clusters=2)




Run experiment

In [ ]:
from kmeans_poc import ExperimentRunner

experiment_runner = ExperimentRunner(
    model=my_model,
    X_train=x,
    y_train=None,
    X_test=x,
    y_test=None,
    data_loader=data_loader,
    log_experiment=True,
    experiment_logger=experimentation,
    evaluator=evaluator,
    experiment_name="/Experiment",
)

results = experiment_runner.run()
print(results)
